In [ ]:
!pip install huggingface_hub
from huggingface_hub import HfApi

api = HfApi()

api.upload_large_folder(
    folder_path=".",
    # path_in_repo="",
    repo_id="adamkarvonen/kl_finetunes",
    repo_type="model",
    allow_patterns=["*ae.pt", "*config.json"],
    ignore_patterns=[".DS_Store"],
)

In [ ]:
import json
from pathlib import Path

k=20
dim = 65536
# ---------------- Configuration Variables ----------------
child_prefix = f"trainer_{k}"          # Prefix for child folders (change as needed)
new_folder_name = "from_scratch_2pow16"  # New name for the current folder

# ---------------- Step 1: Rename the current folder ----------------
# Assumes the current working directory is the folder "trainer_20".
current_folder = Path.cwd()  # e.g. .../gemma-2-2b/trainer_20
parent_folder = current_folder.parent  # e.g. .../gemma-2-2b
new_folder_path = parent_folder / new_folder_name

print(f"Renaming current folder '{current_folder.name}' to '{new_folder_name}'")
current_folder.rename(new_folder_path)

# ---------------- Step 2: Prepend prefix to each child folder ----------------
# Iterate over each subfolder in the newly renamed folder.
for child in new_folder_path.iterdir():
    if child.is_dir():
        # Construct the new child folder name with the desired prefix.
        new_child_name = f"{child_prefix}_{child.name}"
        new_child_path = new_folder_path / new_child_name
        print(f"Renaming folder '{child.name}' to '{new_child_name}'")
        child.rename(new_child_path)

# ---------------- Step 3: Add config.json to each child folder ----------------
# Define the JSON configuration.
config_data = {
    "trainer": {
        "trainer_class": "TopKTrainer",
        "dict_class": "AutoEncoderTopK",
        "lr": 0.0003,
        "steps": 244140,
        "auxk_alpha": 0.03125,
        "warmup_steps": 1000,
        "decay_start": 195312,
        "threshold_beta": 0.999,
        "threshold_start_step": 1000,
        "seed": 0,
        "activation_dim": 2304,
        "dict_size": dim,
        "k": k,
        "device": "cuda:0",
        "layer": 12,
        "lm_name": "google/gemma-2-2b",
        "wandb_name": "TopKTrainer-google/gemma-2-2b-resid_post_layer_12_trainer_20",
        "submodule_name": "resid_post_layer_12"
    },
    "buffer": {
        "d_submodule": 2304,
        "io": "out",
        "n_ctxs": 244,
        "ctx_len": 1024,
        "refresh_batch_size": 4,
        "out_batch_size": 2048,
        "device": "cuda:0"
    }
}

# Add the config.json file to each child folder.
for folder in new_folder_path.iterdir():
    if folder.is_dir():
        config_file = folder / "config.json"
        print(f"Adding config.json to folder '{folder.name}'")
        with open(config_file, "w") as f:
            json.dump(config_data, f, indent=4)

print("All operations completed!")